Script for the dmrg reference of H2^-

In [1]:
from pyscf import gto, scf, mcscf
import numpy as np
import matplotlib.pyplot as plt
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import os 

Use this to delete the temporary files before each dmrg calculation

In [ ]:
folder_path = "./tmp"  #change here the directory where the tmp files are located

for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory) and delete it
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

UHF calculation

In [4]:
H2m = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', charge = -1, spin = 1)
uhf=scf.UHF(H2m).run()

H2_alpha_coeff=uhf.mo_coeff[0]
H2_beta_coeff=uhf.mo_coeff[1]
H2_alpha_en=uhf.mo_energy[0]
H2_beta_en=uhf.mo_energy[1]
H2_alpha_occ=uhf.mo_occ[0]
H2_beta_occ=uhf.mo_occ[1]
print('Number of alpha coefficients:', len(H2_alpha_coeff))
print('Alpha orbital energies: ', H2_alpha_en)
print('Number of beta coefficients:', len(H2_beta_coeff))
print('Beta orbital energies: ', H2_beta_en)
print('Alpha occupation: ', H2_alpha_occ)
print('Beta occupation: ', H2_beta_occ)



converged SCF energy = -0.687169754954451  <S^2> = 0.75  2S+1 = 2
Number of alpha coefficients: 2
Alpha orbital energies:  [-0.02232399  0.20313175]
Number of beta coefficients: 2
Beta orbital energies:  [0.21168621 0.77851178]
Alpha occupation:  [1. 1.]
Beta occupation:  [1. 0.]


DMRG reference calculation

In [ ]:
norbs, nelecas = (2, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf, norbs, nelecas).mc1step()[0]

bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf,
    ncore=0, ncas=None, g2e_symm=1)


driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy)

converged SCF energy = -0.687169754954451  <S^2> = 0.75  2S+1 = 2
UCASSCF energy = -0.687169754954451
UCASCI E = -0.687169754954451  E(CI) = -1.02422530331114  S^2 = 0.7500000
integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =         52

Build MPO | Nsites =     2 | Nterms =         52 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     2 .. Mmpo =    22 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     2 .. Mmpo =     1 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.002 Tmvc-total = 0.000 MPO bond dimension =    22 MaxDW = 0.00e+00
NNZ =           44 SIZE =           44 SPT = 0.0000

Rank =     0 Ttotal =      0.006 MPO method = FastBipartite bond dimension =      22 NNZ =           44 SIZE =           44 SPT = 0.0000

Sweep =    0 | Direction =  forward | Bond dimension =   10 | Noise =  1.00e-04 | Dav threshold =  1.00e-08
Time elapsed =      0.009 | E =      -0.6871697550 | DW = 0